DELETE
FROM public.fact_kpis
where origin_key = 'linea' and "date" < '2023-7-18' and metric_key in ('txcosts_median_usd', 'txcosts_median_eth')

In [ ]:
from src.db_connector import DbConnector

db_connector = DbConnector()

df = db_connector.get_blockspace_contracts('optimism', 7)

In [ ]:
from src.db_connector import DbConnector
import pandas as pd 

db_connector = DbConnector()

exec_string = '''
    select from_address, count(*)
    from optimism_tx
    group by 1
'''

df = pd.read_sql(exec_string, db_connector.engine.connect())

In [ ]:
## from_address column from bytea to text
df['from_address'] = df['from_address'].apply(lambda x: x.hex() if x is not None else None)
## add 0x to from_address
df['from_address'] = '0x' + df['from_address']

In [ ]:
df.to_csv('local/optimism_block_1-114871709.csv', index=False)

In [ ]:
df

In [ ]:
from src.adapters.mapping import adapter_mapping

print([chain.origin_key for chain in adapter_mapping if chain.in_api == True and 'blockspace' not in chain.exclude_metrics])

## web3 test

In [ ]:
from web3 import Web3
import os

tx = '0xdba1ec7832bc857a4f3624557572f24169dda03501fd2b6d6a529647c9a64c51'
url = "https://rpc.zora.energy/"
#url = f"https://rpc.ankr.com/base/{os.getenv('ANKR_API')}"
w3 = Web3(Web3.HTTPProvider(url))

block = w3.eth.get_block(3913960, full_transactions=True)
receipt = w3.eth.get_transaction_receipt(tx)

In [ ]:
from src.misc.helper_functions import api_post_call
import json
 
payload = {
        "jsonrpc": "2.0",
        "method": "eth_getTransactionReceipt",
        "params": [tx],
        "id": 1
    }
headers = {
    "accept": "application/json",
    "content-type": "application/json"
}

response = api_post_call(url, payload=json.dumps(payload), header=headers)

In [ ]:
block_number_hex = hex(3913960)
payload = {
    "jsonrpc": "2.0",
    "method": "eth_getBlockByNumber",
    "params": [str(block_number_hex), True],
    "id": 1
}
headers = {
    "accept": "application/json",
    "content-type": "application/json"
}

response = api_post_call(url, payload=json.dumps(payload), header=headers)

## JSON creation

In [ ]:
import os
from src.api.json_creation import JSONCreation
from src.db_connector import DbConnector

db_connector = DbConnector()

json_creator = JSONCreation(os.getenv("S3_CF_BUCKET"), os.getenv("CF_DISTRIBUTION_ID"), db_connector, "v1")
## for testing
#json_creator = JSONCreation(os.getenv("S3_CF_BUCKET"), os.getenv("CF_DISTRIBUTION_ID"), db_connector, "test")

#json_creator.create_all_jsons()

df = json_creator.get_all_data()
# json_creator.create_landingpage_json(df)
#json_creator.create_chain_details_jsons(df)
json_creator.create_metric_details_jsons(df)
# json_creator.create_fundamentals_json(df)
#json_creator.create_master_json()
# #json_creator.create_contracts_json()

In [ ]:
import os
from src.db_connector import DbConnector
from src.api.blockspace_json_creation import BlockspaceJSONCreation
db_connector = DbConnector()
api_version = "v1"

blockspace_json_creator = BlockspaceJSONCreation(os.getenv("S3_CF_BUCKET"), os.getenv("CF_DISTRIBUTION_ID"), db_connector, api_version)
blockspace_json_creator.create_blockspace_overview_json()
blockspace_json_creator.create_blockspace_comparison_json()

## Cross Check Data from Explorer

In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_cross_check import AdapterCrossCheck

adapter_params = {
}
load_params = {
    'origin_keys' : None,
    #'origin_keys' : ['zksync_era'] # see all options in adapter_mapping.py 
}

# initialize adapter
ad = AdapterCrossCheck(adapter_params, db_connector)
# extract
df = ad.extract(load_params)
# load
ad.load(df)

##ad.cross_check()

## L2Beat

In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_l2beat import AdapterL2Beat

adapter_params = {
}
load_params = {
    #'origin_keys' : None,
    'origin_keys' : ['loopring'] # see all options in adapter_mapping.py 
}

# initialize adapter
ad = AdapterL2Beat(adapter_params, db_connector)
# extract
df= ad.extract(load_params)
# load
ad.load(df)

## DefiLlama

In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_defillama import AdapterDefiLlama

adapter_params = {
}
load_params = {
    #'origin_keys' : None,
    'origin_keys' : ['loopring'] # see all options in adapter_mapping.py
}

# initialize adapter
ad = AdapterDefiLlama(adapter_params, db_connector)
# extract
df = ad.extract(load_params)
# load
ad.load(df)

## Coingecko

run for projects / chains

In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_coingecko import AdapterCoingecko

adapter_params = {
}
load_params = {
    'load_type' : 'project',
    'metric_keys' : ['price', 'volume', 'market_cap'],
   # 'origin_keys' : None,
    'origin_keys' : ['loopring'], # see all options in adapter_mapping.py
    'days' : 'auto', # auto, max, or a number (as string)
    'vs_currencies' : ['usd', 'eth']
}

# initialize adapter
ad = AdapterCoingecko(adapter_params, db_connector)
# extract
df = ad.extract(load_params)
# load
ad.load(df)

and for imx tokens

In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_coingecko import AdapterCoingecko

adapter_params = {
}
load_params = {
    'load_type' : 'imx_tokens'
}

# initialize adapter
ad = AdapterCoingecko(adapter_params, db_connector)
# extract
df = ad.extract(load_params)
# load
ad.load(df)

## Dune

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_dune import AdapterDune

adapter_params = {
    'api_key' : os.getenv("DUNE_API")
}

load_params = {
    'load_type' : 'metrics',
    'query_names' : ['maa'], ## fundamentals, waa, stables_mcap, rent_paid
    'days' : 5000,
    #'query_names' : None,
    # 'days' : 'auto',
}

# initialize adapter
ad = AdapterDune(adapter_params, db_connector)
# extract
df = ad.extract(load_params)
# upload
#ad.load(df)

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_dune import AdapterDune

adapter_params = {
    'api_key' : os.getenv("DUNE_API")
}

load_params = {
    'load_type' : 'inscriptions',
    'days' : 1300,
    'query_names' : None,
    # 'days' : 'auto',
}

# initialize adapter
ad = AdapterDune(adapter_params, db_connector)
# extract
df = ad.extract(load_params)
# upload
ad.load(df)

## Zettablock

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_zettablock import AdapterZettablock

adapter_params = {
    'api_key' : os.getenv("ZETTABLOCK_API")
}
load_params = {
    #'origin_keys' : ['zksync_era', 'polygon_zkevm'],
    'metric_keys' : ['fees_paid_eth', 'txcosts_median_eth'],
    'days' : 5,
    'origin_keys' : None,
    #'metric_keys' : None,
    # 'days' : 'auto',
}

# initialize adapter
ad = AdapterZettablock(adapter_params, db_connector)
# extract
df = ad.extract(load_params)
# load
ad.load(df)

## RPC raw data load

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_raw_rpc import AdapterRPCRaw

adapter_params = {
    'rpc': 'ankr',
    'api_key' : os.getenv("ANKR_API"),
    'chain' : 'base'
}

load_params = {
    'block_start' : 'auto', ## 'auto' or a block number as int
    #'block_start' : 9137631, ## 'auto' or a block number as int
    'batch_size' : 10,
    'threads' : 5
}

# initialize adapter
ad = AdapterRPCRaw(adapter_params, db_connector)
# extract
ad.extract_raw(load_params)

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_raw_rpc import AdapterRPCRaw

adapter_params = {
    'rpc': 'ankr',
    'api_key' : os.getenv("ANKR_API"),
    'chain' : 'optimism'
}

load_params = {
    'block_start' : 'auto', ## 'auto' or a block number as int
    #'block_start' : 9137631, ## 'auto' or a block number as int
    'batch_size' : 10,
    'threads' : 5
}

# initialize adapter
ad = AdapterRPCRaw(adapter_params, db_connector)
# extract
ad.extract_raw(load_params)

## ZettaBlock raw data load

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_raw_zettablock import AdapterZettaBlockRaw

adapter_params = {
    'api_key' : os.getenv("ZETTABLOCK_API")
}

load_params = {
    #'keys' : ['polygon_zkevm_tx', 'zksync_era_tx'],
    'keys' : ['polygon_zkevm_tx'],
    #'block_start' : 'auto', ## 'auto' or a block number as int
    'block_start' : 8456353, ## 'auto' or a block number as int
}

# initialize adapter
ad = AdapterZettaBlockRaw(adapter_params, db_connector)
# extract
ad.extract_raw(load_params, if_exists = 'ignore')

## Chainbase raw

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_raw_chainbase import AdapterChainbaseRaw

adapter_params = {
    'api_key' : os.getenv("CHAINBASE_API")
}

load_params = {
    'keys' : ['arbitrum_tx'],
    #'block_start' : 'auto', ## 'auto' or a block number as int
    'block_start' : 64900000, ## until 65,570,000
}

# initialize adapter
ad = AdapterChainbaseRaw(adapter_params, db_connector)
# extract
ad.extract_raw(load_params)

## IMX raw data load

In [ ]:
## implement orchestration?

from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_raw_imx import AdapterRawImx

adapter_params = {
    'load_types' : ['withdrawals', 'deposits', 'trades', 'orders_filled', 'transfers', 'mints'],
    'forced_refresh' : 'no',

    #'load_types' : ['orders_filled'],
    #'forced_refresh' : '2023-04-01T00:00:00.000000Z',
}

# initialize adapter
ad = AdapterRawImx(adapter_params, db_connector)
# extract raw (and load raw in case of IMX)
df_raw = ad.extract_raw()

## SQL aggregation

In [ ]:
## Loads currently IMX txcount, daa, fees_paid
## also loads user_base_weekly

from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_sql import AdapterSQL

adapter_params = {
}

load_params = {
    'load_type' : 'metrics', ## usd_to_eth or metrics or blockspace
    
    #'days' : 'auto', ## days as int our 'auto
    'origin_keys' : None, ## origin_keys as list or None
    'metric_keys' : None, ## metric_keys as list or None

    'days' : 1500, ## days as int our 'auto
    #'origin_keys' : ['mantle'], ## origin_keys as list or None
    'metric_keys' : ['maa'], ## metric_keys as list or None
}

# initialize adapter
ad = AdapterSQL(adapter_params, db_connector)

# extract
df = ad.extract(load_params)
# # load
ad.load(df)

In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_sql import AdapterSQL

adapter_params = {
}

load_params = {
    'load_type' : 'profit', ## usd_to_eth or metrics or blockspace or profit
    'days' : 30, ## days as int
    #'origin_keys' : ['linea'], ## origin_keys as list or None
    #'origin_keys' : None, ## origin_keys as list or None
    'metric_keys' : None, ## metric_keys as list or None
}

# initialize adapter
ad = AdapterSQL(adapter_params, db_connector)

# extract
df = ad.extract(load_params)
# # load
ad.load(df)

In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_sql import AdapterSQL

adapter_params = {
}

load_params = {
    'load_type' : 'eth_to_usd', ## usd_to_eth or metrics or blockspace
    'days' : 35, ## days as int
    'origin_keys' : ['mantle'], ## origin_keys as list or None
    #'origin_keys' : None, ## origin_keys as list or None
    'metric_keys' : None, ## metric_keys as list or None
}

# initialize adapter
ad = AdapterSQL(adapter_params, db_connector)

# extract
df = ad.extract(load_params)
# # load
#ad.load(df)

In [ ]:
ad.load(df)

In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_sql import AdapterSQL

adapter_params = {
}

load_params = {
    'load_type' : 'blockspace', ## usd_to_eth or metrics or blockspace
    'days' : '75', ## days as or auto
    'origin_keys' : None ## origin_keys as list or None
    #'origin_keys' : ['mantle']
}

# initialize adapter
ad = AdapterSQL(adapter_params, db_connector)

# extract
ad.extract(load_params)

### Blockspace logic
- for each chain, aggregate the daily contracts usage


In [ ]:
from src.db_connector import DbConnector
from src.misc.helper_functions import get_missing_days_blockspace
db_connector = DbConnector()

chain_list = ['arbitrum']
#chain_list = ['arbitrum', 'zksync_era', 'polygon_zkevm', 'optimism']

In [ ]:
for chain in chain_list:
    #days = get_missing_days_blockspace(db_connector, chain)
    days = 75

    # ## aggregate contract data
    # print(f"aggregating contract data for {chain} and last {days} days...")
    # df = db_connector.get_blockspace_contracts(chain, days)
    # df.set_index(['address', 'date', 'origin_key'], inplace=True)

    # print(f"upserting contract data for {chain}. Total rows: {df.shape[0]}...")
    # db_connector.upsert_table('blockspace_fact_contract_level', df)

    # ## determine total usage
    # print(f"aggregating total usage for {chain} and last {days} days...")
    # df = db_connector.get_blockspace_total(chain, days)
    # df.set_index(['date', 'sub_category_key' ,'origin_key'], inplace=True)

    # print(f"upserting total usage usage for {chain}. Total rows: {df.shape[0]}...")
    # db_connector.upsert_table('blockspace_fact_sub_category_level', df)

    # ## aggregate native transfers
    # print(f"aggregating native_transfers for {chain} and last {days} days...")
    # df = db_connector.get_blockspace_native_transfers(chain, days)
    # df.set_index(['date', 'sub_category_key' ,'origin_key'], inplace=True)

    # print(f"upserting native_transfers for {chain}. Total rows: {df.shape[0]}...")
    # db_connector.upsert_table('blockspace_fact_sub_category_level', df)

    ## aggregate contract deployments
    print(f"aggregating smart_contract_deployments for {chain} and last {days} days...")
    df = db_connector.get_blockspace_contract_deplyments(chain, days)
    df.set_index(['date', 'sub_category_key' ,'origin_key'], inplace=True)

    # print(f"upserting smart_contract_deployments for {chain}. Total rows: {df.shape[0]}...")
    # db_connector.upsert_table('blockspace_fact_sub_category_level', df)

    # # ALL below needs to be retriggerd when mapping changes (e.g. new addresses got labeled or new categories added etc.)
    # ## aggregate by sub categories
    # print(f"aggregating sub categories for {chain} and last {days} days...")
    # df = db_connector.get_blockspace_sub_categories(chain, days)
    # df.set_index(['date', 'sub_category_key' ,'origin_key'], inplace=True)

    # print(f"upserting sub categories for {chain}. Total rows: {df.shape[0]}...")
    # db_connector.upsert_table('blockspace_fact_sub_category_level', df)

    # ## determine unlabeled usage
    # print(f"aggregating unlabeled usage for {chain} and last {days} days...")
    # df = db_connector.get_blockspace_unlabeled(chain, days)
    # df.set_index(['date', 'sub_category_key' ,'origin_key'], inplace=True)

    # print(f"upserting unlabeled usage for {chain}. Total rows: {df.shape[0]}...")
    # db_connector.upsert_table('blockspace_fact_sub_category_level', df)


# days = get_missing_days_blockspace(db_connector, 'imx')

# df = db_connector.get_blockspace_imx(days)
# df.set_index(['date', 'sub_category_key' ,'origin_key'], inplace=True)

# print(f"...upserting imx data . Total rows: {df.shape[0]}...")
# db_connector.upsert_table('blockspace_fact_sub_category_level', df)

In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()

In [ ]:
df = db_connector.get_top_contracts_by_category('main_category', 'unlabeled', 'arbitrum', 'gas', 7)

In [ ]:
print(df.to_markdown())

## S3

In [ ]:
## connect to s3 bucket and output list of files
import pandas as pd
import boto3
import os

s3 = boto3.resource(
    service_name='s3',
    region_name='us-east-1',
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")
)

bucket = s3.Bucket('gtp-longterm')

## get list of files in bucket with last modified date
files = []
for obj in bucket.objects.all():
    files.append([obj.key, obj.last_modified])

df = pd.DataFrame(files, columns=['key', 'last_modified'])

## filter out files where key starts with 'imx'
df = df[~df.key.str.startswith('imx')]

df['chain'] = df.key.str.split('/').str[0]

## create new column block_range that extracts the string between 'tx_' and '.parquet' in the key column using lambda function
df['block_range'] = df.key.apply(lambda x: x[x.find('tx_')+3:x.find('.parquet')])
df['block_start'] = df.block_range.str.split('-').str[0].astype(int)
df['block_end'] = df.block_range.str.split('-').str[1].astype(int)

## sort by block_start
df.sort_values(by='block_start', inplace=True, ascending=True)

### Arbitrum

In [ ]:
df_arbitrum = df[df.chain == 'arbitrum']

## load first file in df_arbitrum into df
df_arbitrum_flipside = pd.read_parquet(f"s3://gtp-longterm/{df_arbitrum.key.iloc[0]}")
df_arbitrum_flipside.sort_values(by='ETH_VALUE', inplace=True, ascending=False)
df_arbitrum_flipside.head(10)

In [ ]:
## load first file in df_arbitrum with block_start >= 96528665 into df
df_arbitrum_chainbase = pd.read_parquet(f"s3://gtp-longterm/{df_arbitrum[df_arbitrum.block_start >= 96528665].key.iloc[10]}")
df_arbitrum_chainbase.head()

## indexed.xyz

In [ ]:
import boto3

aws_access_key_id = "43c31ff797ec2387177cabab6d18f15a"
aws_secret_access_key = "afb354f05026f2512557922974e9dd2fdb21e5c2f5cbf929b35f0645fb284cf7"
bucket_name = 'indexed-xyz'

s3 = boto3.client(
        's3',
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key
        )


In [ ]:
import boto3

# Set the profile name
profile_name = 'indexedxyz'

# Create an S3 client using the profile
session = boto3.Session(profile_name=profile_name)
s3 = session.client('s3')

In [ ]:
#test s3 connection
response = s3.list_objects(Bucket=bucket_name)

In [ ]:
## list all files in bucket
for obj in s3.list_objects(Bucket=bucket_name)['Contents']:
    print(obj['Key'])

In [ ]:
from src.adapters.adapter_raw_gtp import NodeAdapter
from src.db_connector import DbConnector
from datetime import datetime,timedelta
from airflow.decorators import dag, task 
import os


adapter_params = {
    'rpc': 'local_node',
    'chain': 'zora',
    'node_url': os.getenv("ZORA_RPC"),
}

# Initialize DbConnector
db_connector = DbConnector()

# Initialize NodeAdapter
adapter = NodeAdapter(adapter_params, db_connector)

# Test database connectivity
if not adapter.check_db_connection():
    print("Failed to connect to database.")
else:
    print("Successfully connected to database.")

# Test S3 connectivity
if not adapter.check_s3_connection():
    print("Failed to connect to S3.")
else:
    print("Successfully connected to S3.")
    
# Extract
load_params = {
    'block_start': '4982120',
    'batch_size': 1,
    'threads': 1,
}
adapter.extract_raw(load_params)



## We <3 the Art

In [ ]:
## load last 100 tx from tx_optimism

from src.db_connector import DbConnector
import pandas as pd
db_connector = DbConnector()

exec_string = """  select to_address, from_address from optimism_tx where to_address is not null order by block_number desc limit 10000 """

df = pd.read_sql(exec_string, db_connector.engine.connect())

## to_address and from_address are bytearrays - convert to string (if not none)
df['to_address'] = df['to_address'].apply(lambda x: x.hex() if x is not None else None)
df['from_address'] = df['from_address'].apply(lambda x: x.hex() if x is not None else None)

In [ ]:
import networkx as nx
import plotly.graph_objects as go

# Assuming df is your DataFrame with 'from_address' and 'to_address' columns
G = nx.from_pandas_edgelist(df, 'from_address', 'to_address')

# Get positions for the nodes in G
pos = nx.spring_layout(G)

# Create Edges
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(x=edge_x, y=edge_y, line=dict(width=0.2, color='#ff0420'), hoverinfo='none', mode='lines')

# Create Nodes
node_x = [pos[node][0] for node in G.nodes()]
node_y = [pos[node][1] for node in G.nodes()]

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        # showscale=True,
        # # colorscale options
        # #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        # #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        # #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        # colorscale='Reds',
        # reversescale=False,
        color="#ff0420",
        size=7,
        line_width=1))

node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append('# of connections: '+str(len(adjacencies[1])))

node_trace.text = node_text

# Create Network Graph
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='Optimism WE <3 THE ART | Just the last 10k Transactions',
                titlefont_size=14,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="@web3_data</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.01, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )

fig.update_layout(
    autosize=False,
    width=800,
    height=800,
    plot_bgcolor='white',
)

fig.show()